# Koopman Linearization of Symbolic Non-Linear Systems

We seek to find Koopman observable for non-linear systems of the symbolic closed form

$$
\dot{\mathbf x} = \begin{bmatrix} f_1(\mathbf x) \\ \vdots \\ f_m(\mathbf x) \end{bmatrix}.
$$

**Lie Derivative.** For a vector field $F = \begin{bmatrix}f_1 \\ f_2 \\ \vdots \\ f_m \end{bmatrix}$, the *Lie Derivative* of a smooth function $f(\mathbf x)$ is given by

$$
\mathcal L_F(f) = \nabla f \cdot F(\mathbf x) = \sum_{i=1}^n \left( \frac{\partial f}{\partial \mathbf x_i} \cdot f_i \right).
$$

## Automatic Abstraction of Symbolic Model [Sankaranarayanan, S. (2011, April)]

1. Start with a sparse, symbolic model defined by the nonlinear equations $\dot {\mathbf x} = F(\mathbf x)$. Choose an initial vector space $V_0 = \operatorname{Span}(\{f_1, \cdots, f_k\})$.
2. At $i$th iteration, refine the basis such that $V_{i+1} \subseteq V_i, \quad V_{i+1} = \{f \in V_i | \mathcal L_F (f) \in V_i\}$.
3. Terminate at iteration $n$ when $V_{n+1}=V_{n}$.

## Data-Driven (eDMD)

1. Start with a sparse, symbolic model defined by the nonlinear equations $\dot {\mathbf x} = F(\mathbf x)$. Choose basis functions $B=\{f_1, \cdots, f_k \}$.
2. Run eDMD on the fixed observables
$$
g(\mathbf x) = \begin{bmatrix} \mathbf x \\ \mathcal L_F(f_1) \\ \vdots \\  \mathcal L_F(f_k) \end{bmatrix}.
$$

## References

Sankaranarayanan, S. (2011, April). Automatic abstraction of non-linear systems using change of bases transformations. In Proceedings of the 14th international conference on Hybrid systems: computation and control (pp. 143-152).


In [ ]:
import sys
sys.path.append("..")

import autokoopman.benchmark.fhn as pfhn
import autokoopman as ak
import sympy as sp

from itertools import product
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
def lie_derivative(f, F, variables):
    """symbolic lie derivative"""
    return sp.expand(sum(sp.diff(f, xi)*F[i] for i, xi in enumerate(variables)))
    
def update_basis(F, basis, variables):
    """abstraction method""" 
    # generate the candidate function
    basis_coef = sp.symbols(" ".join([f"c{i}" for i in range(0, len(basis))]))
    f = sum(ci*bi for ci, bi in zip(basis_coef, basis))

    # lie derivative
    lf = lie_derivative(f, F, variables)

    # remove terms that do not belong to span(*F)
    subs = []
    for term in lf.args:
        in_span = False
        for bf in basis:
            if not any([(term / bf).has(xi) for xi in variables]):
                in_span = True
        if not in_span:
            subs.append([term.has(ci) for ci in basis_coef].index(True))
    return [bi for i, bi in enumerate(basis) if i not in subs]

def obs(F, basis, variables):
    """lie derivative observables"""
    return variables + [lie_derivative(bf, F, variables) for bf in basis]

def generate_monomials(variables, order):
    """generate initial basis functions as monomials up to an order"""
    monomials = set()
    # Generate all combinations of powers including and up to the order for each variable
    for powers in product(range(order + 1), repeat=len(variables)):
        if sum(powers) <= order:
            monomial = sp.prod([var**power for var, power in zip(variables, powers)])
            monomials.add(monomial)
    return list(monomials)

In [ ]:
# build lie observables for the FHN system
fhn = pfhn.FitzHughNagumo()
variables = fhn._variables[1:]
exprs = fhn._exprs
x0, x1 = variables
basis = generate_monomials(variables, 2)[1:]
lie_obs = ak.observable.SymbolicObservable(variables, obs(exprs, basis, variables))

In [ ]:
# learn the Koopman operator (eDMD)
training_data = fhn.solve_ivps(
    initial_states=np.random.uniform(low=-2.0, high=2.0, size=(30, 2)),
    tspan=[0.0, 1.0],
    sampling_period=0.1
)

# learn model from data
experiment_results = ak.auto_koopman(
    training_data,          # list of trajectories
    sampling_period=0.1,    # sampling period of trajectory snapshots
    obs_type=lie_obs,         # use Random Fourier Features Observables
    opt="monte-carlo",             # grid search to find best hyperparameters
    n_obs=200,              # maximum number of observables to try
    max_opt_iter=10,       # maximum number of optimization iterations
    grid_param_slices=5,   # for grid search, number of slices for each parameter
    n_splits=5,             # k-folds validation for tuning, helps stabilize the scoring
    normalize=False,
    rank=(1, 200, 40)       # rank range (start, stop, step) DMD hyperparameter
)

In [ ]:
# plot the results on an extrapolation test set
model = experiment_results['tuned_model']

testing_data = fhn.solve_ivps(
    initial_states=np.random.uniform(low=-3.0, high=3.0, size=(30, 2)),
    tspan=[0.0, 1.0],
    sampling_period=0.1
)

# simulate using the learned model
iv = [0.5, 0.1]
prediction_data = model.solve_ivps(
    initial_states=[t.states[0] for t in testing_data],
    tspan=(0.0, 1.0),
    sampling_period=0.1
)

# plot the results
plt.figure(figsize=(8,8))
for idx, trajectory in enumerate(prediction_data):
    plt.plot(*trajectory.states.T, 'r', label='prediction' if idx == 0 else None)
for idx, true_trajectory in enumerate(testing_data):
    plt.plot(*true_trajectory.states.T, 'k', label='ground truth' if idx == 0 else None)
for idx, trajectory in enumerate(training_data):
    plt.plot(*trajectory.states.T, 'g', label='training data' if idx == 0 else None)
plt.legend()
plt.grid()
plt.title("Lie Derivative Observables Extrapolation Test")